In [1]:
import re
import traceback
import librosa
import sklearn
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import warnings
warnings.filterwarnings("ignore")

In [2]:
def extract_feature():
    id = 0 
    path='./static/'
    feature_set = pd.DataFrame() 
    file_data = [f.strip() for f in listdir(path) if isfile(join(path, f))]
    file_data = [x for x in file_data if re.match(r'.*\.mp3',x, re.I) is not None]
    n_songs = len(file_data)
    
    songname_vector = pd.Series()
    tempo_vector = pd.Series()
    duration_vector = pd.Series()
    rms_mean = pd.Series()
    rms_var = pd.Series()
    zcr_mean = pd.Series()
    zcr_var = pd.Series()
    cent_mean = pd.Series()
    cent_var = pd.Series()
    spec_bw_mean = pd.Series()
    spec_bw_var = pd.Series()
    rolloff_mean = pd.Series()
    rolloff_var = pd.Series()
    frame_mean = pd.Series()
    frame_var = pd.Series()
    
    chroma_stft_mean = np.zeros([n_songs,12])
    chroma_stft_var = np.zeros([n_songs,12])
    chroma_cq_mean = np.zeros([n_songs,12])
    chroma_cq_var = np.zeros([n_songs,12])
    chroma_cens_mean = np.zeros([n_songs,12])
    chroma_cens_var = np.zeros([n_songs,12])
    mel_mean = np.zeros([n_songs,128])
    mel_var = np.zeros([n_songs,128])
    mfcc_mean = np.zeros([n_songs,20])
    mfcc_var = np.zeros([n_songs,20])
    mfcc_delta_mean = np.zeros([n_songs,20])
    mfcc_delta_var = np.zeros([n_songs,20])
    
    contrast_mean = np.zeros([n_songs,7])
    contrast_var = np.zeros([n_songs,7])
    poly_mean = np.zeros([n_songs,3])
    poly_var = np.zeros([n_songs,3])
    tonnetz_mean = np.zeros([n_songs,6])
    tonnetz_var = np.zeros([n_songs,6])
    harm_mean = np.zeros([n_songs,12])
    harm_var = np.zeros([n_songs,12])
    perc_mean = np.zeros([n_songs,12])
    perc_var = np.zeros([n_songs,12])
    # --------------------------------------------------------------------------------------
    for file_name in file_data:
        try:
            songname = path + file_name
            y, sr = librosa.load(songname, duration=60)
            S = np.abs(librosa.stft(y))
            # ----------------------------------------------------------------------------------
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
            rms = librosa.feature.rms(y=y)
            zcr = librosa.feature.zero_crossing_rate(y)
            cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
            frames_to_time = librosa.frames_to_time(onset_frames[:20], sr=sr)

            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
            chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
            melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            mfcc_delta = librosa.feature.delta(mfcc)

            contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
            poly_features = librosa.feature.poly_features(S=S, sr=sr, order=2)
            harmonic = librosa.effects.harmonic(y)
            percussive = librosa.effects.percussive(y)
            harm_chroma_stft = librosa.feature.chroma_stft(y=harmonic, sr=sr)
            perc_chroma_stft = librosa.feature.chroma_stft(y=percussive, sr=sr)
            tonnetz = librosa.feature.tonnetz(y=harmonic, sr=sr)
            # ----------------------------------------------------------------------------------
            songname_vector.at[id] = file_name 
            tempo_vector.at[id] = tempo 
            duration_vector.at[id] = librosa.get_duration(filename=songname) 
            rms_mean.at[id] = np.mean(rms) 
            rms_var.at[id] = np.var(rms)
            zcr_mean.at[id] = np.mean(zcr) 
            zcr_var.at[id] = np.var(zcr)
            cent_mean.at[id] = np.mean(cent) 
            cent_var.at[id] = np.var(cent)
            spec_bw_mean.at[id] = np.mean(spec_bw) 
            spec_bw_var.at[id] = np.var(spec_bw)
            rolloff_mean.at[id] = np.mean(rolloff) 
            rolloff_var.at[id] = np.var(rolloff)
            frame_mean.at[id] = np.mean(frames_to_time) 
            frame_var.at[id] = np.var(frames_to_time)

            chroma_stft_mean[id] = np.mean(chroma_stft, axis=1) 
            chroma_stft_var[id] = np.var(chroma_stft, axis=1)
            chroma_cq_mean[id] = np.mean(chroma_cq, axis=1) 
            chroma_cq_var[id] = np.var(chroma_cq, axis=1)
            chroma_cens_mean[id] = np.mean(chroma_cens, axis=1) 
            chroma_cens_var[id] = np.var(chroma_cens, axis=1)
            mel_mean[id] = np.mean(melspectrogram, axis=1) 
            mel_var[id] = np.var(melspectrogram, axis=1)
            mfcc_mean[id] = np.mean(mfcc, axis=1) 
            mfcc_var[id] = np.var(mfcc, axis=1)
            mfcc_delta_mean[id] = np.mean(mfcc_delta, axis=1) 
            mfcc_delta_var[id] = np.var(mfcc_delta, axis=1)

            contrast_mean[id] = np.mean(contrast, axis=1) 
            contrast_var[id] = np.var(contrast, axis=1)
            poly_mean[id] = np.mean(poly_features, axis=1) 
            poly_var[id] = np.var(poly_features, axis=1)
            tonnetz_mean[id] = np.mean(tonnetz, axis=1) 
            tonnetz_var[id] = np.var(tonnetz, axis=1)
            harm_mean[id] = np.mean(harm_chroma_stft, axis=1) 
            harm_var[id] = np.var(harm_chroma_stft, axis=1)
            perc_mean[id] = np.mean(perc_chroma_stft, axis=1) 
            perc_var[id] = np.var(perc_chroma_stft, axis=1)
            # ----------------------------------------------------------------------------------
            id = id+1
            print('COMPLETED: ',id,'/',n_songs,' : ',file_name)
        except:
            id = id+1
            print('#########   ERROR: ',id,'/',n_songs,' : ',file_name)
            traceback.print_exc()
        
    # --------------------------------------------------------------------------------------
    feature_set['song_name'] = songname_vector
    feature_set['tempo'] = tempo_vector
    feature_set['duration'] = duration_vector
    feature_set['rms_mean'] = rms_mean
    feature_set['rms_var'] = rms_var
    feature_set['zcr_mean'] = zcr_mean
    feature_set['zcr_var'] = zcr_var
    feature_set['cent_mean'] = cent_mean
    feature_set['cent_var'] = cent_var
    feature_set['spec_bw_mean'] = spec_bw_mean
    feature_set['spec_bw_var'] = spec_bw_var
    feature_set['rolloff_mean'] = rolloff_mean
    feature_set['rolloff_var'] = rolloff_var
    feature_set['frame_mean'] = frame_mean
    feature_set['frame_var'] = frame_var

    chroma_stft_mean = pd.DataFrame(data=chroma_stft_mean, columns = ['chroma_stft_mean'+format(a,'02') for a in range(1, 1+chroma_stft_mean.shape[1])])
    chroma_stft_var = pd.DataFrame(data=chroma_stft_var, columns = ['chroma_stft_var'+format(a,'02') for a in range(1, 1+chroma_stft_var.shape[1])])
    chroma_cq_mean = pd.DataFrame(data=chroma_cq_mean, columns = ['chroma_cq_mean'+format(a,'02') for a in range(1, 1+chroma_cq_mean.shape[1])])
    chroma_cq_var = pd.DataFrame(data=chroma_cq_var, columns = ['chroma_cq_var'+format(a,'02') for a in range(1, 1+chroma_cq_var.shape[1])])
    chroma_cens_mean = pd.DataFrame(data=chroma_cens_mean, columns = ['chroma_cens_mean'+format(a,'02') for a in range(1, 1+chroma_cens_mean.shape[1])])
    chroma_cens_var = pd.DataFrame(data=chroma_cens_var, columns = ['chroma_cens_var'+format(a,'02') for a in range(1, 1+chroma_cens_var.shape[1])])
    mel_mean = pd.DataFrame(data=mel_mean, columns = ['mel_mean'+format(a,'03') for a in range(1, 1+mel_mean.shape[1])])
    mel_var = pd.DataFrame(data=mel_var, columns = ['mel_var'+format(a,'03') for a in range(1, 1+mel_var.shape[1])])
    mfcc_mean = pd.DataFrame(data=mfcc_mean, columns = ['mfcc_mean'+format(a,'02') for a in range(1, 1+mfcc_mean.shape[1])])
    mfcc_var = pd.DataFrame(data=mfcc_var, columns = ['mfcc_var'+format(a,'02') for a in range(1, 1+mfcc_var.shape[1])])
    mfcc_delta_mean = pd.DataFrame(data=mfcc_delta_mean, columns = ['mfcc_delta_mean'+format(a,'02') for a in range(1, 1+mfcc_delta_mean.shape[1])])
    mfcc_delta_var = pd.DataFrame(data=mfcc_delta_var, columns = ['mfcc_delta_var'+format(a,'02') for a in range(1, 1+mfcc_delta_var.shape[1])])

    contrast_mean = pd.DataFrame(data=contrast_mean, columns = ['contrast_mean'+format(a,'02') for a in range(1, 1+contrast_mean.shape[1])])
    contrast_var = pd.DataFrame(data=contrast_var, columns = ['contrast_var'+format(a,'02') for a in range(1, 1+contrast_var.shape[1])])
    poly_mean = pd.DataFrame(data=poly_mean, columns = ['poly_mean'+format(a,'02') for a in range(1, 1+poly_mean.shape[1])])
    poly_var = pd.DataFrame(data=poly_var, columns = ['poly_var'+format(a,'02') for a in range(1, 1+poly_var.shape[1])])
    tonnetz_mean = pd.DataFrame(data=tonnetz_mean, columns = ['tonnetz_mean'+format(a,'02') for a in range(1, 1+tonnetz_mean.shape[1])])
    tonnetz_var = pd.DataFrame(data=tonnetz_var, columns = ['tonnetz_var'+format(a,'02') for a in range(1, 1+tonnetz_var.shape[1])])
    harm_mean = pd.DataFrame(data=harm_mean, columns = ['harm_mean'+format(a,'02') for a in range(1, 1+harm_mean.shape[1])])
    harm_var = pd.DataFrame(data=harm_var, columns = ['harm_var'+format(a,'02') for a in range(1, 1+harm_var.shape[1])])
    perc_mean = pd.DataFrame(data=perc_mean, columns = ['perc_mean'+format(a,'02') for a in range(1, 1+perc_mean.shape[1])])
    perc_var = pd.DataFrame(data=perc_var, columns = ['perc_var'+format(a,'02') for a in range(1, 1+perc_var.shape[1])])

    feature_set = pd.concat([feature_set,chroma_stft_mean,chroma_stft_var,chroma_cq_mean,chroma_cq_var,chroma_cens_mean,chroma_cens_var,mel_mean,mel_var,
               mfcc_mean,mfcc_var,mfcc_delta_mean,mfcc_delta_var,contrast_mean,contrast_var,poly_mean,poly_var,tonnetz_mean,tonnetz_var,
               harm_mean,harm_var,perc_mean,perc_var], axis=1)
    # --------------------------------------------------------------------------------------
    feature_set.to_csv('Song_features.csv', mode='a', index=False, header=False)
    

In [7]:
# Extracting Feature Function Call
extract_feature()

COMPLETED:  1 / 386  :  Picture_To_Burn.mp3
COMPLETED:  2 / 386  :  Power.mp3
COMPLETED:  3 / 386  :  Power__feat__Stormzy_.mp3
COMPLETED:  4 / 386  :  Pretend_It_s_OK.mp3
COMPLETED:  5 / 386  :  Priest.mp3
COMPLETED:  6 / 386  :  Private_Show.mp3
COMPLETED:  7 / 386  :  psychofreak__feat__WILLOW_.mp3
COMPLETED:  8 / 386  :  Put_It_All_on_Me__feat__Ella_Mai_.mp3
COMPLETED:  9 / 386  :  Question____.mp3
COMPLETED:  10 / 386  :  Quiet.mp3
COMPLETED:  11 / 386  :  Rainberry.mp3
COMPLETED:  12 / 386  :  Rare.mp3
COMPLETED:  13 / 386  :  Ready_For_Love.mp3
COMPLETED:  14 / 386  :  Ready_For_Ya.mp3
COMPLETED:  15 / 386  :  Ready_to_Run.mp3
COMPLETED:  16 / 386  :  Really.mp3
COMPLETED:  17 / 386  :  Really_Don_t_Care.mp3
COMPLETED:  18 / 386  :  Real_Friends.mp3
COMPLETED:  19 / 386  :  Real_Life_Stuff.mp3
COMPLETED:  20 / 386  :  rEaR_vIeW.mp3
COMPLETED:  21 / 386  :  Red_Desert.mp3
COMPLETED:  22 / 386  :  Red_Line.mp3
COMPLETED:  23 / 386  :  Red_Planet__feat__T-Boz_.mp3
COMPLETED:  24 / 

Traceback (most recent call last):
  File "C:\Users\Harneet\anaconda3\lib\site-packages\librosa\core\audio.py", line 164, in load
    y, sr_native = __soundfile_load(path, offset, duration, dtype)
  File "C:\Users\Harneet\anaconda3\lib\site-packages\librosa\core\audio.py", line 195, in __soundfile_load
    context = sf.SoundFile(path)
  File "C:\Users\Harneet\anaconda3\lib\site-packages\soundfile.py", line 655, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "C:\Users\Harneet\anaconda3\lib\site-packages\soundfile.py", line 1213, in _open
    raise LibsndfileError(err, prefix="Error opening {0!r}: ".format(self.name))
soundfile.LibsndfileError: Error opening './static/Supermarket_Flowers.mp3': File does not exist or is not a regular file (possibly a pipe?).

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-2-3c2fc17e50df>", line 52, in extract_feature
    y, sr = librosa.load(songname, 

COMPLETED:  154 / 386  :  SuperStar.mp3
COMPLETED:  155 / 386  :  Superstar__Taylor_s_Version_.mp3
COMPLETED:  156 / 386  :  Survivors.mp3
COMPLETED:  157 / 386  :  Sweat.mp3
COMPLETED:  158 / 386  :  Sweeter_Than_Fiction_-_From__One_Chance__Soundtrack.mp3
COMPLETED:  159 / 386  :  Sweet_Creature.mp3
COMPLETED:  160 / 386  :  Sweet_Melody.mp3
COMPLETED:  161 / 386  :  Sweet_Melody_-_Karaoke_Version.mp3
COMPLETED:  162 / 386  :  Sweet_Nothing.mp3
COMPLETED:  163 / 386  :  Taken.mp3
COMPLETED:  164 / 386  :  Take_It_Back.mp3
COMPLETED:  165 / 386  :  Take_Me_Back_to_London__feat__Stormzy_.mp3
COMPLETED:  166 / 386  :  Take_My_Hand.mp3
COMPLETED:  167 / 386  :  Take_What_You_Want__feat__5_Seconds_of_Summer_.mp3
COMPLETED:  168 / 386  :  Taki_Taki__with_Selena_Gomez__Ozuna___Cardi_B_.mp3
COMPLETED:  169 / 386  :  Talk_Fast.mp3
COMPLETED:  170 / 386  :  Talk_To_Me.mp3
COMPLETED:  171 / 386  :  Tally.mp3
COMPLETED:  172 / 386  :  TEARS_.mp3
COMPLETED:  173 / 386  :  Teeth.mp3
COMPLETED:  174

COMPLETED:  320 / 386  :  When_Love_s_Around__feat__Syd_.mp3
COMPLETED:  321 / 386  :  When_The_Sun_Goes_Down.mp3
COMPLETED:  322 / 386  :  Wherever_You_Are.mp3
COMPLETED:  323 / 386  :  Where_Do_Broken_Hearts_Go.mp3
COMPLETED:  324 / 386  :  Whiplash.mp3
COMPLETED:  325 / 386  :  WHISTLE.mp3
COMPLETED:  326 / 386  :  White_Christmas.mp3
COMPLETED:  327 / 386  :  White_Horse.mp3
COMPLETED:  328 / 386  :  White_Horse__Taylor_s_Version_.mp3
COMPLETED:  329 / 386  :  wHo.mp3
COMPLETED:  330 / 386  :  Who_Do_You_Love.mp3
COMPLETED:  331 / 386  :  Who_Says.mp3
COMPLETED:  332 / 386  :  Who_s_That_Boy.mp3
COMPLETED:  333 / 386  :  Who_Will_I_Be_-_From__Camp_Rock__Soundtrack_Version.mp3
COMPLETED:  334 / 386  :  Why_Don_t_We_Go_There.mp3
COMPLETED:  335 / 386  :  Why_Won_t_You_Love_Me.mp3
COMPLETED:  336 / 386  :  Wildfire.mp3
COMPLETED:  337 / 386  :  Wildflower.mp3
COMPLETED:  338 / 386  :  willow.mp3
COMPLETED:  339 / 386  :  Windowsill__feat__Devlin_.mp3
COMPLETED:  340 / 386  :  Wings.mp